<a href="https://colab.research.google.com/github/coltongerth/degradation/blob/main/degradation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%%capture
!pip install affine==2.3.1
!pip install attrs==22.2.0
!pip install bounded-pool-executor==0.0.3
!pip install certifi==2022.12.7
!pip install click==8.1.3
!pip install click-plugins==1.1.1
!pip install cligj==0.7.2
!pip install numpy==1.24.1
!pip install python-dateutil==2.8.2
!pip install pytz==2022.7
!pip install rasterio==1.3.4
!pip install scipy==1.10.0
!pip install six==1.16.0
!pip install snuggs==1.4.7
!pip install statsmodels==0.13.5
!pip install git+https://github.com/lankston-consulting/lcutils

Zonal Statistics and Degradation class instantiation.

In [2]:
import concurrent.futures
import os
import numpy as np
import pickle
import rasterio
import asyncio
import warnings
import gc
import mmap
from rasterio.windows import Window
from statsmodels.regression.linear_model import OLS, GLSAR
from scipy import stats as st
from datetime import datetime
from bounded_pool_executor import BoundedProcessPoolExecutor
from dotenv import load_dotenv
from lcutils import gcs, eet
from google.colab import drive



drive.mount('/content/gdrive')
path_to_credentials = "gdrive/MyDrive/fuelcast-storage-credentials.json"
gch = gcs.GcsTools(use_service_account={"keyfile": path_to_credentials})
warnings.filterwarnings("ignore", category=RuntimeWarning)
warnings.filterwarnings("ignore", category=UserWarning)

nodata = -32,768


class Stat(object):
    def __init__(self, zone):
        self.zone = zone
        self.mean = 0
        self.std = 0
        self.n = 0
        self.data = list()

    def add_data(self, data):
        self.data.append(data)

    def calc_stats(self):
        if self.data:
            self.data = np.array(self.data)
            self.data = np.ma.masked_where(self.data < 0, self.data)
            self.mean = np.ma.mean(self.data, axis=0)
            self.std = np.ma.std(self.data, axis=0)
            self.n = np.ma.count(self.data, axis=0)
            self.n = np.ma.masked_where(
                self.n <= 0, self.n
            )

            # Clean up the object to reduce memory footprint
            del self.data


class StatAccumulator(object):
    def __init__(self, update_size=500):
        self.statistics = dict()
        self._update_size = update_size
        self.merged_stats = dict()

    def update(self, zone, new_stats, force=False):
        if zone not in self.statistics:
            self.statistics[zone] = [new_stats]
        else:
            stats_col = self.statistics[zone]
            old_stats = stats_col[-1]

            if len(old_stats.data) > self._update_size or force:
                old_stats.calc_stats()
                self.statistics[zone].append(new_stats)
            else:
                old_stats.data.extend(new_stats.data)
        return

    def update_cochrane(self, zone, new_stats):
        if zone not in self.statistics:
            self.statistics[zone] = new_stats
        else:
            old_stats = self.statistics[zone]

            tn = old_stats.n + new_stats.n
            tmean = (old_stats.n * old_stats.mean + new_stats.n * new_stats.mean) / tn

            t1 = np.ma.add(old_stats.n - 1, np.power(old_stats.std, 2))
            t2 = np.ma.add(new_stats.n - 1, np.power(new_stats.std, 2))
            tr = np.ma.multiply(t1, t2)

            t1 = np.ma.multiply(old_stats.n, new_stats.n)
            t2 = np.ma.add(old_stats.n, new_stats.n)
            tt = np.ma.divide(t1, t2)

            t1 = np.ma.power(old_stats.mean, 2)
            t2 = np.ma.power(new_stats.mean, 2)
            tu = np.ma.add(t1, t2)

            tv = np.ma.multiply(np.ma.multiply(old_stats.mean, new_stats.mean), 2)

            tx = np.ma.add(old_stats.n + new_stats.n - 1, tr)
            xs = np.ma.subtract(tu, tv)
            xt = np.ma.multiply(tt, xs)
            xu = np.ma.add(tx, xt)

            z = np.ma.divide(xu, tx)

            tsd = np.ma.sqrt(z)

            new_stats.n = tn
            new_stats.mean = tmean
            new_stats.std = tsd
            self.statistics[zone] = new_stats

    def update_multiple(self, zone):
        self.statistics[zone][-1].calc_stats()

        def collect(data, tn, tx, txx):
            n = data.n
            mean = data.mean
            sd = data.std
            x = n * mean
            xx = sd**2 * (n - 1) + x**2 / n
            tn = tn + n
            tx = tx + x
            txx = txx + xx

            return tn, tx, txx

        tn, tx, txx = 0, 0, 0
        for data in self.statistics[zone]:
            tn, tx, txx = collect(data, tn, tx, txx)

        tmean = tx / tn
        tsd = np.ma.sqrt(
            np.ma.divide(
                np.ma.subtract(txx, np.ma.divide(np.ma.power(tx, 2), tn)),
                np.ma.add(tn - 1),
            )
        )

        old_stats = self.statistics[zone][0]
        old_stats.mean = tmean
        old_stats.std = tsd
        old_stats.n = tn

        self.merged_stats[zone] = old_stats

        return

    def merge(self):
        self.merged_stats = dict()

        def chunk_gen(lst, n):
            for i in range(0, len(lst), n):
                yield lst[i : i + n]

        def merge_chunk(indexes):
            key_list = list(self.statistics.keys())
            for i in indexes:
                zone = key_list[i]
                self.update_multiple(zone)

        [self.update_multiple(z) for z in self.statistics]
        self.statistics = self.merged_stats
        del self.merged_stats
        return

    def write(self, path="./output/zone_stats.csv"):

        with open(path, "w") as f:
            header = "zone, year, mean, std, n\n"
            f.write(header)

            for z in self.statistics:
                data = self.statistics[z]
                for i in range(len(data.mean)):
                    line = "{0}, {1}, {2}, {3}, {4}\n".format(
                        z, i, data.mean[i], data.std[i], data.n[i]
                    )
                    f.write(line)


class ZonalStatistics(object):
    def __init__(self):

        return

    def data_collector(self, *args, **kwargs):

        stats = dict()

        I = args[0]["zone_data"].shape[0]
        J = args[0]["zone_data"].shape[1]
        K = args[0]["zone_data"].shape[2]

        for i in range(I):
            for j in range(J):
                for k in range(K):
                    zone = args[0]["zone_data"][i, j, k]
                    data = args[0]["val_data"][:, j, k]

                    if zone > 0:
                        if zone not in stats:
                            stats[zone] = Stat(zone)
                        stats[zone].add_data(data)

        return stats

    def t_test(self, *args, **kwargs):

        I = args[0]["zone_data"].shape[0]
        J = args[0]["zone_data"].shape[1]
        K = args[0]["zone_data"].shape[2]

        output = np.empty((4, J, K), dtype="float32")
        output.fill(nodata)

        for i in range(I):
            for j in range(J):
                for k in range(K):
                    zone = args[0]["zone_data"][i, j, k]
                    data = args[0]["val_data"][:, j, k]

                    if zone > 0:
                        stat = args[0]["statistics"].statistics[zone]
                        try:
                            vals = self._t_test_strict_r_logic(stat, data)
                            if vals is not None:
                                output[:, j, k] = vals
                        except Exception as ex:
                            pass

        return output

    def _t_test_strict_r_logic(self, stat, data):
        data = np.ma.masked_where(data < 0, data)
        i_mean = np.ma.mean(data)

        if np.ma.is_masked(i_mean):
            return None

        nan_data = data.astype(float).filled(np.nan)

        p_n_adj = stat.n - 1
        p_mean_list = ((stat.mean * p_n_adj) - data) / p_n_adj
        p_mean_list = np.ma.masked_where(p_mean_list < 0, p_mean_list)

        t, p = st.ttest_rel(nan_data, p_mean_list, nan_policy="omit")

        if np.ma.is_masked(t):
            return None

        years = np.array(list(range(len(p_mean_list))))

        mask_years = np.ma.array(years.astype(float), mask=p_mean_list.mask)
        nan_years = mask_years.filled(np.nan)
        pop_trend_model = GLSAR(p_mean_list, nan_years, missing="drop")
        pop_trend_result = pop_trend_model.fit()

        mask_years = np.ma.array(years.astype(float), mask=data.mask)
        nan_years = mask_years.filled(np.nan)
        ind_trend_model = GLSAR(data, nan_years, missing="drop")
        ind_trend_result = ind_trend_model.fit()

        pop_slope = pop_trend_result.params[0]
        ind_slope = ind_trend_result.params[0]

        slope_diff = ind_slope - pop_slope
        slope_se = ind_trend_result.bse
        slope_t = slope_diff / slope_se

        df = ind_trend_result.nobs - len(ind_trend_result.params)

        slope_p = st.t.sf(np.abs(slope_t), df=df) * 2

        vals = np.array([t, p, slope_t[0], slope_p[0]])

        return vals


class Degradation(object):
    def __init__(self, *args, **kwargs):
        return

    def degradation(self, data):

        I = data.shape[0]
        J = data.shape[1]
        K = data.shape[2]
        output = np.empty((I, J, K))

        for i in range(I):
            for j in range(J):
                for k in range(K):
                    val = data[i, j, k]
                    output[i, j, k] = val
        return output


Mounted at /content/gdrive


Inputs

In [3]:
zone_name = "BpsZonRobGb_wgs84_nc"
gcs_degradation_path = "gs://fuelcast-data/degradation/"
gcs_rpms_path = "gs://fuelcast-data/rpms/"

zone_raster_path = f"{gcs_degradation_path}{zone_name}/{zone_name}.tif"
data_raster_path = f"./data/{zone_name}/rpms_stack.tif"
dummy_path = "./test.tif"

In [ ]:
out_path = [
    f"./output/{zone_name}_mean_t.tif",
    f"./output/{zone_name}_mean_p_adj.tif",
    f"./output/{zone_name}_slope_t.tif",
    f"./output/{zone_name}_slope_p_adj.tif",
]

if not os.path.exists("./output/"):
    os.makedirs("./output/")

BLOCKSIZE = 1024

nodata = -32768


def main_process(task, zone_file, data_file, out_file, queue_size=1):
    deg = Degradation()
    func = deg.degradation

    with rasterio.Env():
        with rasterio.open(zone_file) as zone_src:
            profile = zone_src.profile
            profile.update(blockxsize=BLOCKSIZE, blockysize=BLOCKSIZE, tiled=True)

            with rasterio.open(out_file, "w", **profile) as dst:
                with rasterio.open(data_file) as data_src:

                    zone_windows = [window for ij, window in dst.block_windows()]
                    window_count = len(zone_windows)
                    print("Window Count:", window_count)

                    with BoundedProcessPoolExecutor(max_workers=queue_size) as executor:
                        def stream():
                            yield from iter(zone_windows)

                        def read_zone_ds(window):
                            return zone_src.read(window=window)

                        def read_data(z_window):
                            bounds = zone_src.window_bounds(z_window)
                            data_window = data_src.window(*bounds)
                            return data_src.read(window=data_window)

                        streamer = stream()
                        futures = set()
                        futures_and_windows = dict()

                        for w in streamer:
                            for i in range(queue_size - len(futures)):
                                try:
                                    window = next(streamer)
                                    ex = executor.submit(
                                        func, read_zone_ds(window), read_data(window)
                                    )
                                    futures_and_windows[ex] = window
                                    futures.add(ex)
                                except StopIteration:
                                    pass

                            ex = executor.submit(func, read_zone_ds(w), read_data(w))
                            futures_and_windows[ex] = w
                            futures.add(ex)

                            done, futures = concurrent.futures.wait(
                                futures, return_when=concurrent.futures.FIRST_COMPLETED
                            )

                            for future in done:
                                data = future.result()
                                window = futures_and_windows[future]

                                window_count -= 1
                                print(
                                    f"Remaining: {window_count} || {window} || Size of futures: {len(futures)}"
                                )

                                dst.write(data, window=window)
                                del futures_and_windows[future]

                        done, futures = concurrent.futures.wait(
                            futures, return_when=concurrent.futures.ALL_COMPLETED
                        )

                        for future in done:
                            data = future.result()
                            window = futures_and_windows[future]
                            dst.write(data, window=window)
                            del futures_and_windows[future]
    return


def main_statistics(task, zone_file, data_file, out_files, queue_size=3, batch_size=5, *args, **kwargs):
    zs = ZonalStatistics()

    if task == "collect":
        accumulator = StatAccumulator()
        func = zs.data_collector
    elif task == "degradation":
        if "acc" in kwargs:
            accumulator = kwargs["acc"]
        else:
            raise ValueError()
        func = zs.t_test

    with rasterio.Env():
        with rasterio.open(zone_file) as zone_src:
            profile = zone_src.profile
            profile.update(
                blockxsize=BLOCKSIZE,
                blockysize=BLOCKSIZE,
                tiled=True,
                dtype="int16",
                compress="DEFLATE",
                nodata=nodata,
            )

            if task == "degradation":
                mean_t_raster = rasterio.open(out_files[0], "w", **profile)
                mean_p_raster = rasterio.open(out_files[1], "w", **profile)
                slope_t_raster = rasterio.open(out_files[2], "w", **profile)
                slope_p_raster = rasterio.open(out_files[3], "w", **profile)

            dummy = rasterio.open(dummy_path, "w", **profile)

            with rasterio.open(data_file) as data_src:
                zone_windows = [window for ij, window in dummy.block_windows()]
                window_count = len(zone_windows)
                print("Window Count:", window_count)

                with BoundedProcessPoolExecutor(max_workers=queue_size) as executor:
                    def stream():
                        yield from iter(zone_windows)

                    def read_zone_ds(z_window):
                        return zone_src.read(window=z_window)

                    def read_data(z_window):
                        bounds = zone_src.window_bounds(z_window)
                        d_window = data_src.window(*bounds)
                        return data_src.read(window=d_window), d_window

                    streamer = stream()
                    futures = set()
                    futures_and_windows = dict()

                    def process_future(future):
                        data = future.result()
                        window = futures_and_windows[future]

                        if task == "collect":
                            [accumulator.update(zone, data[zone]) for zone in data]
                        else:
                            data = [
                                data[i, :, :].reshape(
                                    1, data.shape[1], data.shape[2]
                                )
                                for i in range(4)
                            ]
                            mean_t_raster.write(data[0], window=window)
                            mean_p_raster.write(data[1], window=window)
                            slope_t_raster.write(data[2], window=window)
                            slope_p_raster.write(data[3], window=window)

                        del futures_and_windows[future]
                        gc.collect()

                    for w in streamer:
                        if len(futures) >= queue_size:
                            done, futures = concurrent.futures.wait(
                                futures, return_when=concurrent.futures.FIRST_COMPLETED
                            )

                            for future in done:
                                process_future(future)

                        zone = read_zone_ds(w)
                        data, data_window = read_data(w)

                        func_args = {"zone_data": zone, "val_data": data}

                        if task == "degradation":
                            func_args["statistics"] = accumulator

                        ex = executor.submit(func, func_args)
                        futures.add(ex)
                        futures_and_windows[ex] = w

                        if len(futures) >= batch_size:
                            done, futures = concurrent.futures.wait(
                                futures, return_when=concurrent.futures.FIRST_COMPLETED
                            )
                            for future in done:
                                process_future(future)

                    done, futures = concurrent.futures.wait(
                        futures, return_when=concurrent.futures.ALL_COMPLETED
                    )

                    for future in done:
                        process_future(future)

                    if task == "collect":
                        print("Merging collected statistics")
                        accumulator.merge()

                        print("Writing statistics to file")
                        accumulator.write()

                    else:
                        print("uploading")
                        mean_t_raster.close()
                        mean_p_raster.close()
                        slope_t_raster.close()
                        slope_p_raster.close()
                        dummy.close()

                        os.remove(dummy_path)

        return accumulator


def raster_stacker(id, in_ds, out_ds, bounds, chunk_size=1024):
    with rasterio.open(in_ds, lock=False) as src_ds:
        # Calculate the window for the given bounds
        # [-109.061279,36.976227,-102.052002,40.988192]
        win = src_ds.window(
            left=bounds[0],
            bottom=bounds[1],
            right=bounds[2],
            top=bounds[3],

        )

        # win = src_ds.window(
        #     bottom=bounds.bottom,
        #     right=bounds.right,
        #     top=bounds.top,
        #     left=bounds.left,
        # )

        # Convert window dimensions to integers
        win_width = int(win.width)
        win_height = int(win.height)

        # Calculate the number of chunks needed for the window
        num_chunks_x = (win_width + chunk_size - 1) // chunk_size
        num_chunks_y = (win_height + chunk_size - 1) // chunk_size

        for i in range(num_chunks_x):
            for j in range(num_chunks_y):
                col_off = int(win.col_off) + i * chunk_size
                row_off = int(win.row_off) + j * chunk_size
                width = min(chunk_size, win_width - i * chunk_size)
                height = min(chunk_size, win_height - j * chunk_size)

                # Ensure the chunk window is within the raster bounds
                if col_off < 0 or col_off >= src_ds.width:
                    print(f"Skipping chunk ({i}, {j}): col_off out of range col_off={col_off}")
                    continue
                if row_off < 0 or row_off >= src_ds.height:
                    print(f"Skipping chunk ({i}, {j}): row_off out of range row_off={row_off}")
                    continue
                if col_off + width > src_ds.width:
                    width = src_ds.width - col_off
                if row_off + height > src_ds.height:
                    height = src_ds.height - row_off

                if width <= 0 or height <= 0:
                    print(f"Skipping chunk ({i}, {j}): Invalid dimensions col_off={col_off}, row_off={row_off}, width={width}, height={height}")
                    continue

                chunk_win = Window(
                    col_off=col_off,
                    row_off=row_off,
                    width=width,
                    height=height,
                )

                # Log the details of the chunk being processed
                print(f"Processing chunk ({i}, {j}): col_off={col_off}, row_off={row_off}, width={width}, height={height}")

                data = src_ds.read(1, window=chunk_win)

                # Adjust the chunk window to fit within the output dataset dimensions
                out_col_off = col_off
                out_row_off = row_off
                out_width = min(data.shape[1], out_ds.width - out_col_off)
                out_height = min(data.shape[0], out_ds.height - out_row_off)

                if out_col_off + out_width > out_ds.width or out_row_off + out_height > out_ds.height:
                    print(f"Skipping chunk ({i}, {j}): Output window out of range out_col_off={out_col_off}, out_row_off={out_row_off}, width={out_width}, height={out_height}")
                    continue

                out_win = Window(
                    col_off=out_col_off,
                    row_off=out_row_off,
                    width=out_width,
                    height=out_height
                )

                out_ds.write(data, window=out_win, indexes=id)
                print(f"Processed chunk ({i}, {j})")


async def main_run():
    with rasterio.Env(GDAL_NUM_THREADS="ALL_CPUS", verbose=2, GOOGLE_APPLICATION_CREDENTIALS=os.getenv("GOOGLE_APPLICATION_CREDENTIALS", path_to_credentials)):
        zone_ds = rasterio.open(zone_raster_path, chunks=(1024, 1024))
        bounds = zone_ds.bounds
        bounds = [-114.174042,42.633959,-112.858429,43.608239]
        profile = zone_ds.profile
        profile.update(
            blockxsize=1024,
            blockysize=1024,
            tiled=True,
            compress="DEFLATE",
            predictor=2,
            BIGTIFF="Yes",
            dtype="int16"
        )

        od = f"./data/{zone_name}"
        if not os.path.exists(od):
            os.makedirs(od)

        files = [f"gs://fuelcast-data/rpms/{y}/rpms_{y}.tif" for y in range(1985, 2023) if y != 2012]

        profile.update(count=len(files))

        print("Stacking raster")

        stack_path = f"./data/{zone_name}/rpms_stack.tif"

        if os.path.exists(stack_path):
            print(f"Stacked raster {stack_path} already exists.")
        else:
            with rasterio.open(stack_path, "w", **profile) as dst:
                print(f"out: {dst} || {dst.bounds}")

                for id, layer in enumerate(files, start=1):
                    print(f"in: {layer}")
                    raster_stacker(id, layer, dst, bounds)

        print("Calculating zonal statistics")

        acc = main_statistics(
            "collect", zone_raster_path, data_raster_path, out_path, 60
        )

        print("Running degradation")
        start = datetime.now()
        main_statistics(
            "degradation", zone_raster_path, data_raster_path, out_path, 60, acc=acc
        )
        stop = datetime.now()
        print("Total runtime:", (stop - start).seconds / 60, "minutes")

        print("Finished")

await main_run()

Stacking raster
out: <open DatasetWriter name='./data/BpsZonRobGb_wgs84_nc/rpms_stack.tif' mode='w'> || BoundingBox(left=-123.00035763899075, bottom=32.999762673956695, right=-99.99953377828074, top=50.000289594390246)
in: gs://fuelcast-data/rpms/1985/rpms_1985.tif


Processing chunk (0, 0): col_off=39611, row_off=21434, width=1024, height=1024
Processed chunk (0, 0)
Processing chunk (0, 1): col_off=39611, row_off=22458, width=1024, height=1024
Processed chunk (0, 1)
Processing chunk (0, 2): col_off=39611, row_off=23482, width=1024, height=1024
Processed chunk (0, 2)
Processing chunk (0, 3): col_off=39611, row_off=24506, width=1024, height=543
Processed chunk (0, 3)
Processing chunk (1, 0): col_off=40635, row_off=21434, width=1024, height=1024
Processed chunk (1, 0)
Processing chunk (1, 1): col_off=40635, row_off=22458, width=1024, height=1024
Processed chunk (1, 1)
Processing chunk (1, 2): col_off=40635, row_off=23482, width=1024, height=1024
Processed chunk (1, 2)
Processing chunk (1, 3): col_off=40635, row_off=24506, width=1024, height=543
Processed chunk (1, 3)
Processing chunk (2, 0): col_off=41659, row_off=21434, width=1024, height=1024
Processed chunk (2, 0)
Processing chunk (2, 1): col_off=41659, row_off=22458, width=1024, height=1024
Proce

Processing chunk (0, 0): col_off=39611, row_off=21434, width=1024, height=1024
Processed chunk (0, 0)
Processing chunk (0, 1): col_off=39611, row_off=22458, width=1024, height=1024
Processed chunk (0, 1)
Processing chunk (0, 2): col_off=39611, row_off=23482, width=1024, height=1024
Processed chunk (0, 2)
Processing chunk (0, 3): col_off=39611, row_off=24506, width=1024, height=543
Processed chunk (0, 3)
Processing chunk (1, 0): col_off=40635, row_off=21434, width=1024, height=1024
Processed chunk (1, 0)
Processing chunk (1, 1): col_off=40635, row_off=22458, width=1024, height=1024
Processed chunk (1, 1)
Processing chunk (1, 2): col_off=40635, row_off=23482, width=1024, height=1024
Processed chunk (1, 2)
Processing chunk (1, 3): col_off=40635, row_off=24506, width=1024, height=543
Processed chunk (1, 3)
Processing chunk (2, 0): col_off=41659, row_off=21434, width=1024, height=1024
Processed chunk (2, 0)
Processing chunk (2, 1): col_off=41659, row_off=22458, width=1024, height=1024
Proce

Processing chunk (0, 0): col_off=39611, row_off=21434, width=1024, height=1024
Processed chunk (0, 0)
Processing chunk (0, 1): col_off=39611, row_off=22458, width=1024, height=1024
Processed chunk (0, 1)
Processing chunk (0, 2): col_off=39611, row_off=23482, width=1024, height=1024
Processed chunk (0, 2)
Processing chunk (0, 3): col_off=39611, row_off=24506, width=1024, height=543
Processed chunk (0, 3)
Processing chunk (1, 0): col_off=40635, row_off=21434, width=1024, height=1024
Processed chunk (1, 0)
Processing chunk (1, 1): col_off=40635, row_off=22458, width=1024, height=1024
Processed chunk (1, 1)
Processing chunk (1, 2): col_off=40635, row_off=23482, width=1024, height=1024
Processed chunk (1, 2)
Processing chunk (1, 3): col_off=40635, row_off=24506, width=1024, height=543
Processed chunk (1, 3)
Processing chunk (2, 0): col_off=41659, row_off=21434, width=1024, height=1024
Processed chunk (2, 0)
Processing chunk (2, 1): col_off=41659, row_off=22458, width=1024, height=1024
Proce

Processing chunk (0, 0): col_off=39611, row_off=21434, width=1024, height=1024
Processed chunk (0, 0)
Processing chunk (0, 1): col_off=39611, row_off=22458, width=1024, height=1024
Processed chunk (0, 1)
Processing chunk (0, 2): col_off=39611, row_off=23482, width=1024, height=1024
Processed chunk (0, 2)
Processing chunk (0, 3): col_off=39611, row_off=24506, width=1024, height=543
Processed chunk (0, 3)
Processing chunk (1, 0): col_off=40635, row_off=21434, width=1024, height=1024
Processed chunk (1, 0)
Processing chunk (1, 1): col_off=40635, row_off=22458, width=1024, height=1024
Processed chunk (1, 1)
Processing chunk (1, 2): col_off=40635, row_off=23482, width=1024, height=1024
Processed chunk (1, 2)
Processing chunk (1, 3): col_off=40635, row_off=24506, width=1024, height=543
Processed chunk (1, 3)
Processing chunk (2, 0): col_off=41659, row_off=21434, width=1024, height=1024
Processed chunk (2, 0)
Processing chunk (2, 1): col_off=41659, row_off=22458, width=1024, height=1024
Proce

...

Processing chunk (0, 0): col_off=39611, row_off=21434, width=1024, height=1024
Processed chunk (0, 0)
Processing chunk (0, 1): col_off=39611, row_off=22458, width=1024, height=1024
Processed chunk (0, 1)
Processing chunk (0, 2): col_off=39611, row_off=23482, width=1024, height=1024
Processed chunk (0, 2)
Processing chunk (0, 3): col_off=39611, row_off=24506, width=1024, height=543
Processed chunk (0, 3)
Processing chunk (1, 0): col_off=40635, row_off=21434, width=1024, height=1024
Processed chunk (1, 0)
Processing chunk (1, 1): col_off=40635, row_off=22458, width=1024, height=1024
Processed chunk (1, 1)
Processing chunk (1, 2): col_off=40635, row_off=23482, width=1024, height=1024
Processed chunk (1, 2)
Processing chunk (1, 3): col_off=40635, row_off=24506, width=1024, height=543
Processed chunk (1, 3)
Processing chunk (2, 0): col_off=41659, row_off=21434, width=1024, height=1024
Processed chunk (2, 0)
Processing chunk (2, 1): col_off=41659, row_off=22458, width=1024, height=1024
Proce

Processing chunk (0, 0): col_off=39611, row_off=21434, width=1024, height=1024
Processed chunk (0, 0)
Processing chunk (0, 1): col_off=39611, row_off=22458, width=1024, height=1024
Processed chunk (0, 1)
Processing chunk (0, 2): col_off=39611, row_off=23482, width=1024, height=1024
Processed chunk (0, 2)
Processing chunk (0, 3): col_off=39611, row_off=24506, width=1024, height=543
Processed chunk (0, 3)
Processing chunk (1, 0): col_off=40635, row_off=21434, width=1024, height=1024
Processed chunk (1, 0)
Processing chunk (1, 1): col_off=40635, row_off=22458, width=1024, height=1024
Processed chunk (1, 1)
Processing chunk (1, 2): col_off=40635, row_off=23482, width=1024, height=1024
Processed chunk (1, 2)
Processing chunk (1, 3): col_off=40635, row_off=24506, width=1024, height=543
Processed chunk (1, 3)
Processing chunk (2, 0): col_off=41659, row_off=21434, width=1024, height=1024
Processed chunk (2, 0)
Processing chunk (2, 1): col_off=41659, row_off=22458, width=1024, height=1024
Proce

Processing chunk (0, 0): col_off=39611, row_off=21434, width=1024, height=1024
Processed chunk (0, 0)
Processing chunk (0, 1): col_off=39611, row_off=22458, width=1024, height=1024
Processed chunk (0, 1)
Processing chunk (0, 2): col_off=39611, row_off=23482, width=1024, height=1024
Processed chunk (0, 2)
Processing chunk (0, 3): col_off=39611, row_off=24506, width=1024, height=543
Processed chunk (0, 3)
Processing chunk (1, 0): col_off=40635, row_off=21434, width=1024, height=1024
Processed chunk (1, 0)
Processing chunk (1, 1): col_off=40635, row_off=22458, width=1024, height=1024
Processed chunk (1, 1)
Processing chunk (1, 2): col_off=40635, row_off=23482, width=1024, height=1024
Processed chunk (1, 2)
Processing chunk (1, 3): col_off=40635, row_off=24506, width=1024, height=543
Processed chunk (1, 3)
Processing chunk (2, 0): col_off=41659, row_off=21434, width=1024, height=1024
Processed chunk (2, 0)
Processing chunk (2, 1): col_off=41659, row_off=22458, width=1024, height=1024
Proce

Processing chunk (0, 0): col_off=39611, row_off=21434, width=1024, height=1024
Processed chunk (0, 0)
Processing chunk (0, 1): col_off=39611, row_off=22458, width=1024, height=1024
Processed chunk (0, 1)
Processing chunk (0, 2): col_off=39611, row_off=23482, width=1024, height=1024
Processed chunk (0, 2)
Processing chunk (0, 3): col_off=39611, row_off=24506, width=1024, height=543
Processed chunk (0, 3)
Processing chunk (1, 0): col_off=40635, row_off=21434, width=1024, height=1024
Processed chunk (1, 0)
Processing chunk (1, 1): col_off=40635, row_off=22458, width=1024, height=1024
Processed chunk (1, 1)
Processing chunk (1, 2): col_off=40635, row_off=23482, width=1024, height=1024
Processed chunk (1, 2)
Processing chunk (1, 3): col_off=40635, row_off=24506, width=1024, height=543
Processed chunk (1, 3)
Processing chunk (2, 0): col_off=41659, row_off=21434, width=1024, height=1024
Processed chunk (2, 0)
Processing chunk (2, 1): col_off=41659, row_off=22458, width=1024, height=1024
Proce

Processing chunk (0, 0): col_off=39611, row_off=21434, width=1024, height=1024
Processed chunk (0, 0)
Processing chunk (0, 1): col_off=39611, row_off=22458, width=1024, height=1024
Processed chunk (0, 1)
Processing chunk (0, 2): col_off=39611, row_off=23482, width=1024, height=1024
Processed chunk (0, 2)
Processing chunk (0, 3): col_off=39611, row_off=24506, width=1024, height=543
Processed chunk (0, 3)
Processing chunk (1, 0): col_off=40635, row_off=21434, width=1024, height=1024
Processed chunk (1, 0)
Processing chunk (1, 1): col_off=40635, row_off=22458, width=1024, height=1024
Processed chunk (1, 1)
Processing chunk (1, 2): col_off=40635, row_off=23482, width=1024, height=1024
Processed chunk (1, 2)
Processing chunk (1, 3): col_off=40635, row_off=24506, width=1024, height=543
Processed chunk (1, 3)
Processing chunk (2, 0): col_off=41659, row_off=21434, width=1024, height=1024
Processed chunk (2, 0)
Processing chunk (2, 1): col_off=41659, row_off=22458, width=1024, height=1024
Proce